In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re


In [75]:
train = pd.read_csv("train.csv")


In [108]:
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

시작시간 없는건 최빈값으로 채우기

두 컬럼 둘다 fillna(0) 해주기.

분으로 바꿔서 총 운항시간(분) 확인하기

distance랑 비교해서 값내고 1분에 몇마일 or 1마일에 몇분 컬럼 추가

모든 컬럼값 더해서 평균내기

평균낸걸로 distance 비교해서 채우기



In [109]:
count = test['Estimated_Arrival_Time'].value_counts().get(600.0, 0)
print(count)

410


In [110]:
count = test['Estimated_Departure_Time'].value_counts().get(600.0, 0)
print(count)


14623


In [111]:
for index, row in test.iterrows():
    if pd.isnull(row['Estimated_Arrival_Time']) and pd.isnull(row['Estimated_Departure_Time']):
        test.at[index, 'Estimated_Departure_Time'] = test['Estimated_Departure_Time'].mode().iloc[0]


In [112]:
test['Estimated_Departure_Time'].isnull().sum()

97056

In [113]:
test['Estimated_Arrival_Time'].isnull().sum()

109048

In [114]:
null_mask = test[['Estimated_Departure_Time', 'Estimated_Arrival_Time']].isnull().all(axis=1)
null_rows = test[null_mask]

print(f"{len(null_rows)}")


0


In [115]:
test['Estimated_Departure_Time'] = test['Estimated_Departure_Time'].fillna(0)
test['Estimated_Departure_Time'] = test['Estimated_Departure_Time'].astype(int)

test['Estimated_Arrival_Time'] = test['Estimated_Arrival_Time'].fillna(0)
test['Estimated_Arrival_Time'] = test['Estimated_Arrival_Time'].astype(int)

In [116]:
def convert_to_time(minutes):
    hours = minutes // 60
    minutes = minutes % 60
    return f"{hours:02d}{minutes:02d}"

In [117]:
def convert_to_minutes(time):    
    return int(time // 100) * 60 + int(time % 100)

In [118]:
test['Flight_Time'] = None
test['Estimated_Departure_Minutes'] = None
test['Estimated_Arrival_Minutes'] = None

for index, row in test.iterrows():
    dep_time = row['Estimated_Departure_Time']
    arr_time = row['Estimated_Arrival_Time']
    
    if arr_time == 0 and dep_time == 0:
        continue
        
    elif dep_time == 0:
        arr_time_in_minutes = convert_to_minutes(arr_time)
        test.at[index, 'Estimated_Arrival_Minutes'] = arr_time_in_minutes
        
    elif arr_time == 0:
        dep_time_in_minutes = convert_to_minutes(dep_time)
        test.at[index, 'Estimated_Departure_Minutes'] = dep_time_in_minutes
        
    else:
        dep_time_in_minutes = convert_to_minutes(dep_time)
        arr_time_in_minutes = convert_to_minutes(arr_time)
        
        if arr_time_in_minutes < dep_time_in_minutes:
            arr_time_in_minutes += 1440
        
        duration_in_minutes = arr_time_in_minutes - dep_time_in_minutes
        test.at[index, 'Flight_Time'] = duration_in_minutes
        test.at[index, 'Estimated_Arrival_Minutes'] = arr_time_in_minutes
        test.at[index, 'Estimated_Departure_Minutes'] = dep_time_in_minutes


In [119]:
for index, row in test.iterrows():
    if pd.isnull(row['Flight_Time']):
        test.at[index, 'Flight_Time_per_Distance'] = None
    else:
        flight_time_per_distance = row['Flight_Time'] / row['Distance']
        test.at[index, 'Flight_Time_per_Distance'] = flight_time_per_distance

In [120]:
avg_list = []
for index, row in test.iterrows():
    if pd.isnull(row['Flight_Time_per_Distance']):
        pass
    else:
        avg_list.append(row['Flight_Time_per_Distance'])
 

In [121]:
print(sum(avg_list), len(avg_list))

183699.93423152083 793896


In [122]:
velocity_avg = sum(avg_list) / len(avg_list)
velocity_avg

0.23139042674546897

In [123]:
t_f = []
for i in avg_list:
    if i > velocity_avg:
        t_f.append('big')
    else:
        t_f.append('small')

In [124]:
print(t_f.count('small'), t_f.count('big'))

498876 295020


In [125]:
for index, row in test.iterrows():
    if row['Flight_Time'] is None:
        test.at[index, 'Flight_Time'] = row['Distance'] * velocity_avg
    else:
        pass

In [126]:
test['Flight_Time'].isnull().sum()

0

In [127]:
test.columns

Index(['ID', 'Month', 'Day_of_Month', 'Estimated_Departure_Time',
       'Estimated_Arrival_Time', 'Cancelled', 'Diverted', 'Origin_Airport',
       'Origin_Airport_ID', 'Origin_State', 'Destination_Airport',
       'Destination_Airport_ID', 'Destination_State', 'Distance', 'Airline',
       'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Tail_Number', 'tail_alpha',
       'Flight_Time', 'Estimated_Departure_Minutes',
       'Estimated_Arrival_Minutes', 'Flight_Time_per_Distance'],
      dtype='object')

In [128]:
test = test.drop(['ID',  'Day_of_Month', 
       'Estimated_Arrival_Time', 'Cancelled', 'Diverted', 'Origin_Airport',
        'Origin_State', 'Destination_Airport',
        'Destination_State',  'Airline',
       'Carrier_Code(IATA)',  'Tail_Number', 
       'tail_alpha', 'Estimated_Departure_Minutes',
       'Estimated_Arrival_Minutes', 'Flight_Time_per_Distance'], axis=1)

In [129]:
test.to_csv("test_1.csv", index=False)

In [131]:
train

,Month,Estimated_Departure_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Delay,Flight_Time
0,4,600,13851,12191,419.0,19393.0,NaN,97.601139
1,8,740,13930,14869,1250.0,20304.0,NaN,164
2,9,1610,11057,12953,544.0,19805.0,NaN,115
3,7,905,12892,11618,2454.0,19977.0,NaN,510
4,1,900,14771,10157,250.0,20304.0,NaN,79
...,...,...,...,...,...,...,...,...
999995,9,936,13930,14100,678.0,19977.0,NaN,187
999996,5,920,11637,13487,223.0,20304.0,NaN,68
999997,6,800,13796,12191,1642.0,19393.0,NaN,340
999998,9,1613,10693,10397,214.0,19790.0,NaN,131


In [130]:
test

,Month,Estimated_Departure_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Flight_Time
0,12,1156,12266,14683,191.0,19977.0,44.195572
1,9,1500,11618,10397,746.0,19790.0,135
2,3,1600,13930,12953,733.0,19977.0,195
3,5,1920,13796,12892,337.0,19393.0,85
4,7,1915,11697,12892,2343.0,20409.0,157
...,...,...,...,...,...,...,...
999995,6,0,12892,11292,862.0,19393.0,199.458548
999996,6,600,10792,13930,473.0,19977.0,48
999997,6,1710,12264,11433,383.0,20378.0,117
999998,11,0,14679,10721,2588.0,20409.0,598.838424
